In [5]:
## packages installed

import numpy as np
import pandas as pd
import sklearn
import seaborn
import keras
import mediapipe as mp 

### Load Data 

In [6]:
## TO-DO: Load the ASL Alphabet dataset, using keras.preprocessing.image 

from keras.preprocessing.image import ImageDataGenerator



### MediaPipe

##### check https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python for guidance on this section

In [ ]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [7]:
## TO-DO: Download and Load Model Path

## Example model path given here 

model_path = '/absolute/path/to/gesture_recognizer.task'


In [ ]:
## TO-DO: prepare image data, create task, and run the task

import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode



In [ ]:
def process_output(landmarks):
    """
    extract and clean landmark coordinate data: including 
    - Removing depth (or z dimension)
    - Centralizing coordinates to center point of hand
    - Flattening to 1D array 
    - Normalization w.r.t. max value 
 
    Paramaters
    ---------
    landmakrs: output from Mediapipe model 
    """
    ##TO-DO: complete and use this function!

In [ ]:

##TO-DO: generate the train and test set
X_train, y_train, X_test, y_test = ...

### Model Training with ResNet / VGG

In [9]:
##TO-DO: train and evaluate model

### Model Training with Mediapipe Landmarks (Random Forest, LSTM, etc.)

In [8]:
##TO-DO: train and evaluate model